# SUSA CX Kaggle Capstone Project
## Part 3: Hyperparameter tuning, Decision Trees, Ensemble Learning

### Table Of Contents
* [Introduction](#section1)
* [Hyperparameters](#section2)
* [GridSearch](#section2i)
* [Decision Trees](#section3)   
* [Random Forest](#section4)
* [Ensemble learning](#section5)
* [Conclusion](#conclusion)
* [Additional Reading](#reading)


### Hosted by and maintained by the [Statistics Undergraduate Students Association (SUSA)](https://susa.berkeley.edu). Originally authored by [Patrick Chao](mailto:prc@berkeley.edu), & [Noah Gundotra](mailto:noah.gundotra@berkeley.edu).


<a id='section1'></a>
# SUSA CX Kaggle Capstone Project

Welcome to the second week of the SUSA CX Kaggle Capstone! Now that you have an understanding of your data, a cleaned dataset, and a working model, now it's time to improve your model performance with some new tricks and techniques.  This week, you and your teammates will choose one of three techniques to improve your model further, read an article on the topic, and read through documentation to write the model for yourselves! The overarching goal of this week is to give you practice reading about and coding basic, but powerful, models - just like you'd do in real life when learning a new model design. 
# Logistics

Most of the logistics are the same as last week, but we are repeating them here for your convenience. Please let us know if you or your teammates are feeling nervous about the pace of this project - remember that we are not grading you on your project, and we really try to make the notebooks relatively easy and fast to code through. If for any reason you are feeling overwhelmed or frustrated, please DM us or talk to us in person. We want all of you to have a productive, healthy, and fun time learning data science!

## hope you enjoyed datathon

### Mandatory Office Hours

Because this is such a large project, you and your team will surely have to work on it outside of meetings. In order to get you guys to seek help from this project, we are making it **mandatory** for you and your group to attend **two (2)** SUSA Office Hours over the next 4 weeks. This will allow questions to be answered outside of the regular meetings and will help promote collaboration with more experienced SUSA members.

The schedule of SUSA office hours are below:
https://susa.berkeley.edu/calendar#officehours-table

We understand that most of you will end up going to Arun or Patrick's office hours, but we highly encourage you to go to other people's office hours as well. There are many qualified SUSA mentors who can help and this could be an opportunity for you to meet them.

In [2]:
# import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

# Hyperparameters and Tuning

# Grid Search

# Decision Trees

# Random Forest

# Ensemble Learning
----

Ensemble learning is one of the most widely used techniques to improved model accuracy. Many automated, programmatic frameworks for simple modeling tasks lend themselves to being ensembled. 

Ensembling is the process of training multiple models to do the same thing, and averaging their predictions. Ensemble models often train their models on different (overlapping) splits of the dataset. These datasets are picked randomly _**with**_ replacement. Intuitively, you can imagine training the same type of linear model and giving each model only 1 category of data to learn from. It's possible to imagine that each linear model as becoming an expert in its dataset, and can contribute unique information to other expert models. From high level perspective, ensembling is merging the expert opinions of different models.

From this narrative, we can see how very different models produce a much more useful ensemble than identical models. Clearly, having 10 modes that detects numbers 0-9 is more useful to digit detection than having 10 models which all detect the number 1. This is known as independence of errors. 

Along this note... There are good formal **statistical** and **mathematical** reasons to do ensembling. One of the easier things to prove about ensembling is that averaging model predictions actually reduces variance on error.

We just wanted to introduce you to what the math looks like, in case you're interested. The following is from the Deep Learning Book (Goodfellow 2016). Given $k$ models, with errors $\epsilon_i$ on each example, and errors with variance $v$, the expected loss of the model decreases as more models are ensembled. Specifically, this works when the correlation between the models, $c$, is less than 1.

$$
\begin{align}
\mathbb{E}\left[\left(\frac{1}{k} \sum_i \epsilon_i\right)^2\right] &= \frac{1}{k^2}\mathbb{E}\left[\sum_{i}\left(\epsilon_i^2 + \sum_{j\not=i} \epsilon_i\epsilon_j\right)\right] \\
&= \frac{1}{k}v + \frac{k-1}{k}c
\end{align}
$$

A second good reason to use ensembling is to improve the space of representations that the aggregate (total) model can learn. Understanding this line of reasoning requires the ability to visualize a model's decision boundary. Essentially, multiple single models can overfit to the data, creating complex, abstract hyperplanes to separate between classifications.

![Ensemble of overfit models](GRAPHICS/overfitboundary.png)

If you're interested in reading up on this, and learning where that figure came from, check out [MLWave's explanation.](https://mlwave.com/kaggle-ensembling-guide/)
#### Do not worry about the math!

We do not need to understand it to understand how ensembling works. We'll begin our practical walk through below.


# Conclusion

# Additional Reading